ライブラリのインストール

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1
!pip xformers==0.0.28.post3 ipywidgets==8.1.5 peft==0.13.2
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Using cached accelerate-1.5.2-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.5.2-py3-none-any.whl (345 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.4.0
    Uninstalling accelerate-1.4.0:
      Successfully uninstalled accelerate-1.4.0


In [ ]:
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

モデルのパス

In [2]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# データとモデルのパスを設定（適宜変更）
DATA_FILE_PATH = '/content/drive/MyDrive/program/code_translation/data/sample_data.jsonl'
MODEL_PATH = '/content/drive/MyDrive/program/code_translation/base_model/llm-jp-3-1.8b'
OUTPUT_DIR = '/content/drive/MyDrive/program/code_translation/model/test_model_1'

# wandbのプロジェクト名
WANDB_PROJECT = 'code_translation'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Weights & Biasesにログイン

In [3]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: puwaer (puwaer-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

loraのパラメータ

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 512                    # コンテキスト長を設定
dtype = None                            # Noneにしておけば自動
load_in_4bit = True                     # 量子化

model_id = "llm-jp/llm-jp-3-13b"
new_model_id = "llm-jp-3-13b-it-lora_" #Fine-Tuningしたモデルにつけたい名前、it: Instruction Tuning
# FastLanguageModel インスタンスを作成
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

# SFT用のモデルを用意
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, #デフォルト32
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,                                            #デフォルト32
    lora_dropout = 0.05,                                        #デフォルト0.05
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,                                        #デフォルト3407
    use_rslora = False,
    loftq_config = None,
    max_seq_length = max_seq_length,
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Converting train dataset to ChatML:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [ ]:
# 学習時のプロンプトフォーマットの定義
prompt = """### 指示
{}
### 回答
{}"""


"""
formatting_prompts_func: 各データをプロンプトに合わせた形式に合わせる
"""
EOS_TOKEN = tokenizer.eos_token # トークナイザーのEOSトークン（文末トークン）
def formatting_prompts_func(examples):
    input = examples["text"] # 入力データ
    output = examples["output"] # 出力データ
    text = prompt.format(input, output) + EOS_TOKEN # プロンプトの作成
    return { "formatted_text" : text, } # 新しいフィールド "formatted_text" を返す
pass

# # 各データにフォーマットを適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc= 4, # 並列処理数を指定
)

dataset

学習のパラメータ

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    max_seq_length = max_seq_length,
    dataset_text_field="formatted_text",
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        logging_steps = 10,
        warmup_steps = 10,
        save_steps=100,
        save_total_limit=2,
        max_steps=-1,
        learning_rate = 1.75e-4, #デフォルト2e-4
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        group_by_length=True,
        seed = 3407, #デフォルト3407
        output_dir = "outputs",
        report_to = "none",
    ),
)

学習と保存

In [ ]:
trainer_stats = trainer.train()


model.push_to_hub_merged(
    new_model_id+"_lora",
    tokenizer=tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    private=True
)